In [1]:
import numpy as np
import pandas as pd
import os
import tifffile
import shutil
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from skimage import measure
from skimage import io

In [18]:
def plot_surface(filepath):
    """
    This function calculates the surfaces (using marching cubes), and displays them in 3D

    """

    img = tifffile.imread(filepath)

    # use marching cubes to extract the surface, play around with the level value to see which level yields the best results
    verts, faces, _, _ = measure.marching_cubes(img, level = 42)

    # plot
    fig = go.Figure(data = [go.Mesh3d(x = verts[:, 0], y = verts[:, 1], z = verts[:, 2],
                                     i = faces[:, 0], j = faces[:, 1], k = faces[:, 2],
                                     opacity = 0.5)])

    fig.update_layout(scene = dict(xaxis = dict(title='X'),
                                 yaxis = dict(title='Y'),
                                 zaxis = dict(title='Z')),
                      margin = dict(l = 0, r  =0, t = 0, b = 0),
                      title = filepath)

    fig.show()

In [ ]:
 # display one volume in 3D
filepath = "bcrick_1_000.tif"
img = tifffile.imread(filepath)

fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(111, projection = '3d')

ax.voxels(img, alpha = 0.4, cmap = 'magma')

plt.show()

In [ ]:
filepath1 = "bcrick_1_000.tif"
filepath2 = "soldat_1_000.tif"
plot_surface(filepath2)

In [19]:
filepath2 = "bcrick_1_000.tif"
filepath1 = "soldat_1_000.tif"
filepath3 = "output_directory8/slice_0000.tif"
plot_surface(filepath2)
plot_surface(filepath1)

In [81]:

def load_volume_from_slices(directory_path):
       # List all TIFF files in the directory, sorted to maintain the correct order
    slices = sorted([os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.endswith('.tif')])

    # Read each slice as an array and stack them along a new axis to form a 3D volume
    volume = np.stack([tifffile.imread(slice) for slice in slices], axis=0)

    return volume


Loaded volume dimensions: (128, 64, 64)


In [80]:
from scipy.ndimage import zoom

def resize_volume(volume, target_shape):
    """
    Resize a 3D volume to a new shape using interpolation.

    Parameters:
        volume (numpy.ndarray): The original 3D volume array.
        target_shape (tuple): The target dimensions (z, y, x).

    Returns:
        numpy.ndarray: The resized volume.
    """
    # Calculate the zoom factors for each dimension
    zoom_factors = [new_dim / old_dim for new_dim, old_dim in zip(target_shape, volume.shape)]

    # Perform the resizing
    resized_volume = zoom(volume, zoom_factors, order=1)  # order=1 (bilinear interpolation) is usually a good balance
    return resized_volume

Original volume shape: (256, 64, 64)
Resized volume shape: (128, 64, 64)


In [83]:
def read_volume(filepath):
    """
    Read a TIFF file and return its contents as a numpy array.
    """
    return tifffile.imread(filepath)

def merge_volumes(volume1, volume2, axis=0):
    """
    Merge two 3D volumes along a specified axis after checking if other dimensions are compatible.
    """
    if volume1.shape != volume2.shape:
        raise ValueError("Volumes must have the same dimensions for a max merge.")
    return np.concatenate((volume1, volume2), axis=axis)

def save_volume_as_tiff(volume, output_dir):
    """
    Save each slice of a 3D volume as a separate TIFF file in the specified directory.
    """
    # Create the output directory, removing it first if it exists
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir)

    # Save each slice to the output directory
    for i, slice in enumerate(volume):
        tifffile.imwrite(f"{output_dir}/slice_{i:04d}.tif", slice)

def main():
    target_dimensions = (128, 64, 64)
    #merge
    filepath1 = "bcrick_1_000.tif"
    filepath2 = "soldat_1_000.tif"
    volume1 = read_volume(filepath1)
    volume2 = read_volume(filepath2)
    merged_volume = merge_volumes(volume1, volume2, axis=1)
    output_dir = 'output_directory7'
    save_volume_as_tiff(merged_volume, output_dir)
    #resize
    volume7 = load_volume_from_slices('output_directory7')
    resized_volume = resize_volume(volume7, target_dimensions)
    output_dir = 'output_directory7resized'
    save_volume_as_tiff(resized_volume, output_dir)
    print("Original volume shape:", volume7.shape)
    print("Resized volume shape:", resized_volume.shape)

    #merge
    filepath2 = "bcrick_1_000.tif"
    filepath1 = "soldat_1_000.tif"
    volume1 = read_volume(filepath1)
    volume2 = read_volume(filepath2)
    merged_volume = merge_volumes(volume1, volume2, axis=1)
    output_dir = 'output_directory8'
    save_volume_as_tiff(merged_volume, output_dir)
    #resize
    volume8 = load_volume_from_slices('output_directory8')
    resized_volume = resize_volume(volume8, target_dimensions)
    output_dir = 'output_directory8resized'
    save_volume_as_tiff(resized_volume, output_dir)
    print("Original volume shape:", volume8.shape)
    print("Resized volume shape:", resized_volume.shape)

    #merge
    filepath1 = "output_directory7resized"
    filepath2 = "output_directory8resized"
    volume1 = load_volume_from_slices(filepath1)
    volume2 = load_volume_from_slices(filepath2)
    merged_volume = merge_volumes(volume1, volume2, axis=0)
    output_dir = 'output_directory9'
    save_volume_as_tiff(merged_volume, output_dir)
    #resize
    volume9 = load_volume_from_slices('output_directory9')
    resized_volume = resize_volume(volume9, target_dimensions)
    output_dir = 'output_directory9resized'
    save_volume_as_tiff(resized_volume, output_dir)
    print("Original volume shape:", volume8.shape)
    print("Resized volume shape:", resized_volume.shape)

main()

Original volume shape: (128, 128, 64)
Resized volume shape: (128, 64, 64)
Original volume shape: (128, 128, 64)
Resized volume shape: (128, 64, 64)
Original volume shape: (128, 128, 64)
Resized volume shape: (128, 64, 64)


In [84]:
def plot_surface(volume):
    # Use marching cubes to extract the surface
    verts, faces, _, _ = measure.marching_cubes(volume, level=42)

    # Plot
    fig = go.Figure(data=[go.Mesh3d(x=verts[:, 0], y=verts[:, 1], z=verts[:, 2],
                                   i=faces[:, 0], j=faces[:, 1], k=faces[:, 2],
                                   opacity=0.5)])
    fig.update_layout(scene=dict(xaxis=dict(title='X'),
                                  yaxis=dict(title='Y'),
                                  zaxis=dict(title='Z')),
                      margin=dict(l=0, r=0, t=0, b=0))
    fig.show()




# Use the function with the loaded volume
plot_surface(load_volume_from_slices('output_directory9resized'))